In [ ]:
from jupyter_plotly_dash import JupyterDash


import dash_core_components as dcc
import dash_html_components as html
import dash
from dash.dependencies import Input, Output
from pymongo import MongoClient
import urllib.parse
from bson.json_util import dumps
#TODO: import for their CRUD module
from crud import AnimalShelter
# image encoder
import base64


# this is a juypter dash application
app = JupyterDash('ModuleFive')

image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# the application interfaces are declared here
# this application has two input boxes, a submit button, a horizontal line and div for output
app.layout = html.Div(
    [
    html.Center([
        # customer image location with anchor tag to the client’s home page: www.snhu.edu.
        html.A([
            html.Img(id='customer-image',
                     src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     alt='Grazioso Salvare Logo',
                     style={'width': 225})
        ], href="https://www.arsari.us", target="_blank"),
        #TODO: insert unique identifier code here
        html.H1("Animal Shelter Search Dashboard"),
        html.H5("Developed by Arturo Santiago-Rivera", style={'color': 'green'}),
        ]),
        html.Hr(),
        
        # user sign-in input fields
        html.Div(
            [
                ('Sign In: '),
                dcc.Input(
                    id="input_user".format("text"),
                    type="text",
                    placeholder="input {}".format("username")),
                dcc.Input(
                    id="input_passwd".format("password"),
                    type="password",
                    placeholder="input {}".format("password")),
                dcc.Input(
                    id="input_db".format("text"),
                    type="text",
                    placeholder="input {}".format("database")),
                html.Button(
                    'Execute',
                    id='submit-val',
                    n_clicks=0,
                    style={'background-color': 'darkblue', 'color': 'white', 'padding': '5px 10px', 'font-weight': 'bold'})
            ]),
        html.Hr(),

        # querys output
        html.Div([
                html.P("Future Content", style={'fontSize': 30})
            ]),
        
        # footer
        html.Div(
            [
                html.Hr(),
                html.P(
                    [
                        "Module 5-2 Assignment: Client-Side Authentication - Prof. Tad Kellogg M.S.",
                        html.Br(),
                        "CS-340-T3237 Client/Server Development 21EW3 - Southern New Hampshire University",
                        html.Br(),
                        "February 7, 2021"
                    ], style={'fontSize': 12})
            ])
    ]
)

# this is area to define application responses or callback routines
# this one callback will take the entered text and if the submit button is clicked then call the 
#  mongo database with the find_one query and return the result to the output div
@app.callback(
    Output("query-out", "children"),
    [Input("input_user".format("text"), "value"), 
     Input("input_passwd".format("password"),"value"),
     Input("input_db".format("text"), "value"),
     Input('submit-val', 'n_clicks')],
    [dash.dependencies.State('input_passwd', 'value')]
)
def cb_render(userValue,passValue,dbValue,n_clicks,buttonValue):
    if n_clicks > 0:
        ###########################
        # Data Manipulation / Model
        # use CRUD module to access MongoDB
        ##########################
        username = urllib.parse.quote_plus(userValue)
        password = urllib.parse.quote_plus(passValue)
        dbname = urllib.parse.quote_plus(dbValue)        
        #TODO: Instantiate CRUD object with above authentication username and password values
        user = AnimalShelter(username, password, dbname)

        # note that MongoDB returns BSON, the pyMongo JSON utility function dumps is used to convert to text
        #TODO: Return example query results
        # Read method
        read_query = {'animal_type': 'Dog', 'name': 'Lucy'}
        docs = user.read(read_query)
        return html.Div([
            html.H3("Read Method"),
            html.P("Find Documents with query {}".format(read_query)),
            html.P([
                "Total of Documents Found {}".format(docs.count()),
                html.Br(),
                "=================="
            ]),
            html.P(dumps(docs.limit(2)), style={"color": "blue"})
        ])

    
app
